In [1]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px

In [2]:
reader = Reader()

In [3]:
data_folder = join('..', 'data')

In [4]:
dir_list = ['knots_1year', 'knots_3year', 'knots_expert1', 'expert1_no_tps', 'expert1_no_opt', 'expert1_no_tps_no_opt']

In [5]:
knot_time = {}
tps_forces = {}

for d in dir_list:
    knots_dir = [x[0] for x in os.walk(join(data_folder, d))][1:]
    knots_dir.sort()
    knot_time[d] = []
    tps_forces[d] = {}
    for l in ['right', 'left']:
        for f in ['index', 'middle', 'thumb']:
            key = l + '_' + f + '_calibrated'
            tps_forces[d][key] = []
    
    for kd in knots_dir:
        reader.play(kd)
        time = reader.data['camera']['relative_time'].iloc[-1]
        
        print(reader.data)

        knot_time[d].append(time)
        for l in ['right', 'left']:
            for f in ['index', 'middle', 'thumb']:
                key = l + '_' + f + '_calibrated'
                tps_forces[d][key].append(reader.data['tps'][key].to_numpy())
        
# add time for the knots of 1styear not segmented
knot_time['knots_1year'] = [72.6, 76.2, 117.16, 121.8] + knot_time['knots_1year']

2 3852
{'camera':       index  absolute_time  relative_time
3         9   1.600425e+09       0.018840
4        10   1.600425e+09       0.049095
5        11   1.600425e+09       0.081831
6        12   1.600425e+09       0.117558
7        13   1.600425e+09       0.150775
...     ...            ...            ...
3848   3854   1.600425e+09     129.133163
3849   3855   1.600425e+09     129.153003
3850   3856   1.600425e+09     129.176811
3851   3857   1.600425e+09     129.211035
3852   3858   1.600425e+09     129.244764

[3850 rows x 3 columns], 'emg':          index  absolute_time  relative_time        emg0       emg1  \
64         173   1.600425e+09       0.001900   39.552094 -14.770852   
65         174   1.600425e+09       0.002932   14.526917 -15.991593   
66         175   1.600425e+09       0.003964    4.455809 -12.838013   
67         176   1.600425e+09       0.004996  -12.939742  -7.446410   
68         177   1.600425e+09       0.006027  -16.703691  -5.310114   
...        ...     

0 3263
{'camera':       index  absolute_time  relative_time
1      4703   1.600425e+09       0.001486
2      4704   1.600425e+09       0.038690
3      4705   1.600425e+09       0.074894
4      4706   1.600425e+09       0.102175
5      4707   1.600425e+09       0.131935
...     ...            ...            ...
3259   7961   1.600426e+09     108.639431
3260   7962   1.600426e+09     108.673655
3261   7963   1.600426e+09     108.706887
3262   7964   1.600426e+09     108.741112
3263   7965   1.600426e+09     108.774840

[3263 rows x 3 columns], 'emg':          index  absolute_time  relative_time       emg0        emg1  \
22      155957   1.600425e+09       0.002656 -16.093321 -100.324407   
23      155958   1.600425e+09       0.004383  12.594078  -48.036028   
24      155959   1.600425e+09       0.006109   8.931857    0.488402   
25      155960   1.600425e+09       0.007835 -15.381223   69.358505   
26      155961   1.600425e+09       0.009561  -3.682461   89.195537   
...        ...     

1 2898
{'camera':       index  absolute_time  relative_time
2     11731   1.600426e+09       0.026785
3     11732   1.600426e+09       0.058528
4     11733   1.600426e+09       0.094240
5     11734   1.600426e+09       0.127476
6     11735   1.600426e+09       0.160209
...     ...            ...            ...
2894  14623   1.600426e+09      96.396163
2895  14624   1.600426e+09      96.441795
2896  14625   1.600426e+09      96.465604
2897  14626   1.600426e+09      96.496355
2898  14627   1.600426e+09      96.529091

[2897 rows x 3 columns], 'emg':         index  absolute_time  relative_time        emg0       emg1  \
78     387712   1.600426e+09       0.001487  -77.740710  32.838022   
79     387713   1.600426e+09       0.002975 -140.812297  59.694311   
80     387714   1.600426e+09       0.004463 -139.896741  80.955539   
81     387715   1.600426e+09       0.005951  -92.999965  78.208873   
82     387716   1.600426e+09       0.007439  -23.214307  68.748135   
...       ...            

2 3218
{'camera':       index  absolute_time  relative_time
3        10   1.600430e+09       0.018353
4        11   1.600430e+09       0.054067
5        12   1.600430e+09       0.083825
6        13   1.600430e+09       0.117552
7        14   1.600430e+09       0.148800
...     ...            ...            ...
3214   3221   1.600430e+09     107.127129
3215   3222   1.600430e+09     107.161353
3216   3223   1.600430e+09     107.199546
3217   3224   1.600430e+09     107.230297
3218   3225   1.600430e+09     107.262537

[3216 rows x 3 columns], 'emg':          index  absolute_time  relative_time       emg0       emg1       emg2  \
119        277   1.600430e+09       0.002164  15.698057 -41.880197  -4.444159   
120        278   1.600430e+09       0.003493  17.122255 -29.062423  11.527194   
121        279   1.600430e+09       0.004822  18.546452 -20.720697  31.059041   
122        280   1.600430e+09       0.006152 -21.941438 -22.246623  39.502495   
123        281   1.600430e+09       0.00

0 3085
{'camera':       index  absolute_time  relative_time
1      8750   1.600431e+09       0.031376
2      8751   1.600431e+09       0.064609
3      8752   1.600431e+09       0.098336
4      8753   1.600431e+09       0.134049
5      8754   1.600431e+09       0.164801
...     ...            ...            ...
3081  11830   1.600431e+09     102.706407
3082  11831   1.600431e+09     102.738647
3083  11832   1.600431e+09     102.774856
3084  11833   1.600431e+09     102.806103
3085  11834   1.600431e+09     102.839335

[3085 rows x 3 columns], 'emg':          index  absolute_time  relative_time       emg0       emg1       emg2  \
1       288582   1.600431e+09       0.001260  -0.474817  28.314310 -39.538509   
2       288583   1.600431e+09       0.002520  -5.764692  19.362214 -31.095055   
3       288584   1.600431e+09       0.003779 -14.615060  13.055055 -38.114312   
4       288585   1.600431e+09       0.005039 -15.123702  19.565671 -52.661468   
5       288586   1.600431e+09       0.00

...

1 1904
{'camera':       index  absolute_time  relative_time
2     11603   1.600437e+09       0.019839
3     11604   1.600437e+09       0.048111
4     11605   1.600437e+09       0.085315
5     11606   1.600437e+09       0.115567
6     11607   1.600437e+09       0.157727
...     ...            ...            ...
1900  13501   1.600437e+09      63.259872
1901  13502   1.600437e+09      63.294096
1902  13503   1.600437e+09      63.326833
1903  13504   1.600437e+09      63.360559
1904  13505   1.600437e+09      63.393792

[1903 rows x 3 columns], 'emg':         index  absolute_time  relative_time       emg0       emg1       emg2  \
41     382369   1.600437e+09       0.001497 -34.616714   8.719569  -8.777709   
42     382370   1.600437e+09       0.002777 -19.560916 -26.071531  -6.641414   
43     382371   1.600437e+09       0.004056 -18.645361 -28.106099  -9.286351   
44     382372   1.600437e+09       0.005336 -18.340176 -26.681902 -13.253757   
45     382373   1.600437e+09       0.006616 -

0 1976
{'camera':       index  absolute_time  relative_time
1      2159   1.600437e+09       0.019344
2      2160   1.600437e+09       0.051584
3      2161   1.600437e+09       0.086304
4      2162   1.600437e+09       0.118545
5      2163   1.600437e+09       0.152272
...     ...            ...            ...
1972   4130   1.600437e+09      65.735907
1973   4131   1.600437e+09      65.765666
1974   4132   1.600437e+09      65.802867
1975   4133   1.600437e+09      65.833123
1976   4134   1.600437e+09      65.866354

[1976 rows x 3 columns], 'emg':         index  absolute_time  relative_time       emg0       emg1       emg2  \
11      71137   1.600437e+09       0.001120  16.104027   1.251686 -16.957691   
12      71138   1.600437e+09       0.002598  22.818099  -2.919177 -13.397199   
13      71139   1.600437e+09       0.004076  25.869950  -6.886583  -6.072756   
14      71140   1.600437e+09       0.005554  23.326741  -8.615966  24.852667   
15      71141   1.600437e+09       0.007033  

0 1276
{'camera':       index  absolute_time  relative_time
1      1696   1.600437e+09       0.031892
2      1697   1.600437e+09       0.068104
3      1698   1.600437e+09       0.099349
4      1699   1.600437e+09       0.132579
5      1700   1.600437e+09       0.165812
...     ...            ...            ...
1272   2967   1.600437e+09      42.397755
1273   2968   1.600437e+09      42.431483
1274   2969   1.600437e+09      42.462234
1275   2970   1.600437e+09      42.496953
1276   2971   1.600437e+09      42.528698

[1276 rows x 3 columns], 'emg':        index  absolute_time  relative_time        emg0       emg1       emg2  \
1      55856   1.600437e+09       0.001260  124.874417  72.687766   1.274453   
2      55857   1.600437e+09       0.002520  137.590462  19.280374   1.172725   
3      55858   1.600437e+09       0.003779  107.885780 -40.637633   9.616179   
4      55859   1.600437e+09       0.005039   51.731722 -34.025289  11.345562   
5      55860   1.600437e+09       0.006299   

2 1019
{'camera':       index  absolute_time  relative_time
3         8   1.600438e+09       0.005951
4         9   1.600438e+09       0.036208
5        10   1.600438e+09       0.069937
6        11   1.600438e+09       0.100198
7        12   1.600438e+09       0.134417
...     ...            ...            ...
1015   1020   1.600438e+09      33.722561
1016   1021   1.600438e+09      33.757779
1017   1022   1.600438e+09      33.791010
1018   1023   1.600438e+09      33.822754
1019   1024   1.600438e+09      33.858962

[1017 rows x 3 columns], 'emg':        index  absolute_time  relative_time        emg0       emg1       emg2  \
84       181   1.600438e+09       0.001121   15.822602  -2.997146 -16.832203   
85       182   1.600438e+09       0.002341   25.588525  -5.336898 -18.256400   
86       183   1.600438e+09       0.003561   21.214205  -7.066280 -14.085537   
87       184   1.600438e+09       0.004781   17.145071 -11.745785  -9.609489   
88       185   1.600438e+09       0.006002   

1 1470
{'camera':       index  absolute_time  relative_time
2      3771   1.600438e+09       0.008433
3      3772   1.600438e+09       0.041176
4      3773   1.600438e+09       0.072416
5      3774   1.600438e+09       0.102673
6      3775   1.600438e+09       0.136400
...     ...            ...            ...
1466   5235   1.600438e+09      48.805433
1467   5236   1.600438e+09      48.838666
1468   5237   1.600438e+09      48.874875
1469   5238   1.600438e+09      48.909595
1470   5239   1.600438e+09      48.939354

[1469 rows x 3 columns], 'emg':         index  absolute_time  relative_time       emg0       emg1       emg2  \
48     124290   1.600438e+09       0.001289  16.839886 -21.816893 -22.935905   
49     124291   1.600438e+09       0.002648  15.415688 -16.425290 -25.173929   
50     124292   1.600438e+09       0.004007  19.586551 -12.050970 -11.745785   
51     124293   1.600438e+09       0.005366  21.010749 -13.475167  -3.607516   
52     124294   1.600438e+09       0.006725  

# Execution time

In [ ]:
for key, value in knot_time.items():
    print(key, np.mean(value), np.std(value))

In [ ]:
knot_time

## Differences between subjects

In [ ]:
stats.ttest_ind(knot_time['knots_1year'], knot_time['knots_3year'])

In [ ]:
stats.ttest_ind(knot_time['knots_1year'], knot_time['knots_expert1'])

In [ ]:
stats.ttest_ind(knot_time['knots_3year'], knot_time['knots_expert1'])

No differences between 1st and 3rd year interns, difference beween them and expert

## Differences with and without sensors

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][-3:], knot_time['expert1_no_tps'][-3:])

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][-3:], knot_time['expert1_no_opt'][-3:])

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][-3:], knot_time['expert1_no_tps_no_opt'][-3:])

In [ ]:
stats.ttest_rel(knot_time['expert1_no_tps'][-3:], knot_time['expert1_no_opt'][-3:])

In [ ]:
stats.ttest_rel(knot_time['expert1_no_tps'][-3:], knot_time['expert1_no_tps_no_opt'][-3:])

In [ ]:
stats.ttest_rel(knot_time['expert1_no_opt'][-3:], knot_time['expert1_no_tps_no_opt'][-3:])

No significant difference in terms of execution time

## Adaptation

In [ ]:
print(np.mean(knot_time['knots_expert1'][:4]), np.std(knot_time['knots_expert1'][:4]))
print(np.mean(knot_time['knots_expert1'][4:8]), np.std(knot_time['knots_expert1'][4:8]))
print(np.mean(knot_time['knots_expert1'][8:12]), np.std(knot_time['knots_expert1'][8:12]))

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][:4], knot_time['knots_expert1'][4:8])

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][4:8], knot_time['knots_expert1'][8:12])

In [ ]:
stats.ttest_rel(knot_time['knots_expert1'][:4], knot_time['knots_expert1'][8:12])

No sign of adatptation for expert

In [ ]:
stats.ttest_rel(knot_time['knots_1year'][:4], knot_time['knots_1year'][-4:])

In [ ]:
stats.ttest_rel(knot_time['knots_3year'][:3], knot_time['knots_3year'][-3:])

No sign of adatptation for any of them

# TPS Data

In [ ]:
tps_forces